In [ ]:
# load data
# abhi

from matplotlib import pyplot as plt
import numpy as np
import os
import pickle
from tqdm import tqdm

x = []
x_load = []
y = []
y_load = []


def load_data():
    count = 0
    for file in tqdm(os.listdir("/home/abhi/mainproject/quickdraw/data")):
        file = "/home/abhi/mainproject/quickdraw/data//" + file
        print(file)
        x = np.load(file)
        x = x.astype('float32') / 255.
        x = x[0:10000, :]
        x_load.append(x)
        y = [count for _ in range(10000)]
        count += 1
        y = np.array(y).astype('float32')
        y = y.reshape(y.shape[0], 1)
        y_load.append(y)

    return x_load, y_load


features, labels = load_data()

features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

# shape[0] number numpy files
# shape[1] 10000
features=features.reshape(features.shape[0]*features.shape[1],features.shape[2])
# number of files * 10000 *784
# print(features.shape)

labels=labels.reshape(labels.shape[0]*labels.shape[1],labels.shape[2])
# c=0
# d=0
# f=0
# for i in labels:
#     if (i ==2) and (c==0): 
#         print ("two")
#         c=1

#     if i==1 and d==0:
#         print("one")
#         d=1
#     if i==0 and f==0:
#         print("zero")
#         f=1

with open("features", "wb") as f:
    pickle.dump(features, f, protocol=2)
with open("labels", "wb") as f:
    pickle.dump(labels, f, protocol=2)


In [ ]:
# Data loading manupulation
# image from dataset printing
# abhi
import numpy as np
from matplotlib import pyplot as plt
import os 
files = "/home/abhi/mainproject/quickdraw/data/rainbow.npy"
x = np.load(files)
print(x[0].shape)
img=np.array(x[1000])
img=np.reshape(img,(28,28))
print(img.shape)
# print(img)
plt.imshow(img,cmap="gray")
plt.show()

In [ ]:
# modelcreation
# abhi

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt


def keras_model(image_x, image_y):
    num_of_classes = 8
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels



def prepress_labels(labels):
    labels = np_utils.to_categorical(labels)
    return labels


def main():
    features, labels = loadFromPickle()
    features, labels = shuffle(features, labels)
    labels=prepress_labels(labels)
#     print ("labels prepress",labels[1])
    train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,                                                     test_size=0.1)
    print("shape",test_x.shape)
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)
    print("shape1",test_x.shape)
#     print("train_x[1]",train_x[0])
    model, callbacks_list = keras_model(28,28)
    print_summary(model)
    H=model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64,
              callbacks=[TensorBoard(log_dir="QuickDraw")])
    
    model.save('QuickDraw.h5')
    plt.style.use("ggplot")
    plt.figure()
    N = 3
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig("plot")


main()


In [ ]:
import cv2
from keras.models import load_model
import numpy as np
from matplotlib import pyplot as plt
import tkinter
from tkinter import messagebox

window=tkinter.Tk()
width=400
height=200
screen_width=window.winfo_screenwidth()
screen_height=window.winfo_screenheight()
x_cord=(screen_width/2)-(width/2)
y_cord=(screen_height/2)-(height/2)
window.geometry("%dx%d+%d+%d" % (width,height,x_cord,y_cord))
window.withdraw()
def keras_process_image(img):
    image_x = 28
    image_y = 28
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

test1=cv2.imread("test/envelope.jpg",0)

model=load_model("QuickDraw.h5")

print("before processs",test1.shape)
plt.imshow(test1,cmap="gray")
plt.show()

digit=cv2.bitwise_not(test1)
processed = keras_process_image(digit)
print("new image",processed.shape)

print("processed: " + str(processed.shape))
pred_probab = model.predict(processed)[0]
print("prob",pred_probab)
pred_class = list(pred_probab).index(max(pred_probab))
print("class",pred_class)
if pred_class ==0 :
    print("the doodle is star")
    result=messagebox.askyesno("Prediction","did u draw a envelope")
    
elif pred_class ==1:
    print("the doodle is Envelope")
elif pred_class==2:
    print("the doodle is star")
window.mainloop()